# Import lib

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
np.random.seed(42)
tf.random.set_seed(42)

# Add data

In [5]:
pos_df = pd.read_csv('reviews-data/reviews-data/positive_data.csv')
pos_df.head()

,Rate,Review,Label
0,9.0,Khu ẩm thực với đa dạng đồ lại còn bày trí đẹ...,1
1,9.0,Lúc nào đến aeon là lúc đấy phải tống một đốn...,1
2,10.0,Bánh ngon lại rẻ chê đâu được gần hết các loạ...,1
3,9.0,Ngon rẻ,1
4,9.6,Tôi sắp chết vì ngập trong sushi mấttttt Lên ...,1


In [6]:
neg_df = pd.read_csv('reviews-data/reviews-data/negative_data.csv')
neg_df.head()

,Rate,Review,Label
0,4.0,Mình thề là mình ko thể cảm nổi đồ ăn ở aeon ...,-1
1,3.8,Đôi khi thèm lên là bất chấp nắng nóng phi Và...,-1
2,3.8,Ngõ treo biển cafe trứng đúng kiểu phố cổ hà ...,-1
3,3.8,Mình thấy địa chỉ cafe Giảng ở Nguyễn Hữu Huâ...,-1
4,2.2,Mình là người Hà Nội và cũng cực kỳ khó tính ...,-1


In [7]:
pos_df["Label"].value_counts()

Label
1    2642
Name: count, dtype: int64

In [8]:
neg_df["Label"].value_counts()

Label
-1    1765
Name: count, dtype: int64

> Một số trường hợp class của sample có tỉ lệ quá khác biệt ảnh hưởng đến chất lượng mô hình. Vì vậy nên sử dụng phương thức chọn mẫu `DataFrame.sample()` để cho số lượng mẫu = nhau.

In [9]:
pos_df_sample = pos_df.sample(n=len(neg_df), random_state=42)

> Kết hợp dữ liệu positive và negative vào cùng 1 DataFrame bằng `pandas.concat`

In [10]:
df = pd.concat([pos_df_sample, neg_df])
df["Label"].value_counts()

Label
 1    1765
-1    1765
Name: count, dtype: int64

In [11]:
#xem dữ liệu mới nhất
df.tail()

,Rate,Review,Label
1760,3.2,Mình đã ăn bò bít tết tại bò dai và chán kinh...,-1
1761,3.2,Ăn không Sốt ít mà khoai mềm được mỗi quả trứ...,-1
1762,2.8,Sự thực là khá tò mò khi thấy bạn bè check in...,-1
1763,3.8,Vị nằm ngay mặt đường nên dễ nhưng gửi xe lại...,-1
1764,4.0,mình uống ở cơ sở thụy trên đg thụy khuê có c...,-1


> Để phân phối lại các sample pos/nev trong df, sử dụng phương thức `DataFrame.sample`

> Ở đây chọn `frac=1` để chọn **tất cả** các sample trong df với thứ tự ngẫu nhiên.

In [13]:
df = df.sample(frac=1).reset_index(drop=True)
df.tail()

,Rate,Review,Label
3525,10.0,Ngon và rất,1
3526,10.0,Món ăn rẻ vừa túi tiền với mọi quý vị thơm kh...,1
3527,2.4,thất vọng toàn ko ngon gì cả,-1
3528,3.4,Khá thất vọng khi ăn ở cơ sở Giảng Võ lần này...,-1
3529,2.2,Vào hỏi menu thì bảo không lúc tính tiền thì ...,-1


> Đầu ra của mô hình phân loại 2 lớp (**binary classification**) là giá trị 0 hoặc 1 => Vì vậy cần `chuyển -1 (nev) -> 0`

In [14]:
df['Label'] = df['Label'].replace({-1:0})
df.tail()

,Rate,Review,Label
3525,10.0,Ngon và rất,1
3526,10.0,Món ăn rẻ vừa túi tiền với mọi quý vị thơm kh...,1
3527,2.4,thất vọng toàn ko ngon gì cả,0
3528,3.4,Khá thất vọng khi ăn ở cơ sở Giảng Võ lần này...,0
3529,2.2,Vào hỏi menu thì bảo không lúc tính tiền thì ...,0


# Chia tập train:test
- Sử dụng thuộc tính `values` của Pandas series để lấy các giá trị dưới dạng **numpy ndarray**
- Các dữ liệu dung để train và test nên được chuyển sang `ndarray` để phù hợp với các hàm tính toán trong ML và DL.

In [15]:
X = df["Review"].values
y = df["Label"].values
X.shape, y.shape

((3530,), (3530,))

> Sử dụng hàm `train_test_split` để chia tập train:test **(tỉ lệ 8:2)**

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# doi kieu du lieu ve numpy array
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((2824,), (706,), (2824,), (706,))

# Tokenize dữ liệu train - test
Vận dụng các kiến thức tokenizing và padding ở buổi trước để xử lý dữ liệu chữ.\
Nhắc lại các bước xử lý:
1. Chọn kích thước từ điển
2. Chọn độ dài lớn nhất của một chuỗi (sequence)
3. Tạo Tokenizer và fit Tokenizer (trên văn bản của tập train)
4. Sử dụng Tokenizer đã huấn luyện để tạo tokens
5. Sử dụng padding và truncating để các chuỗi có độ dài bằng nhau

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

In [20]:
vocab_size = 10000
max_length = 400

In [22]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')

In [23]:
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

In [24]:
print("Length of train dataset:", len(X_train_pad))
print("Length of test dataset:", len(X_test_pad))

Length of train dataset: 2824
Length of test dataset: 706
